# DILEEPKUMAR - AIQoD - ML Assignment
-----------------------------------------------------------------------

The data contains features extracted from text similar to the one shown below:

![image.png](attachment:image.png)


You have to create a ML model that predict the probability that a piece of text belongs to a particular class.
Use techniques like Bag of Words, tfidf vectorization and word embedding. Please use Hash field value and explain how you are going to use the Hash field.

## Data extraction
For the documents nGrams have been extracted, Each row in the Train.csvcorresponds to one such nGram.

## Features
For a given nGram several features have been extracted (145). These features have been saved in the train.csvand test.csv. They have parsing, spatial, content and relative information.

    • Content: The cryptographic hash of the raw text.
    • Parsing: nGram is a number, text, alphanumeric, etc.
    • Spatial: Position and size of the nGram
    • Relational: details of text nearby the nGram

### The feature values can be:
    • Numbers. Continuous/discrete numerical values.
    • Boolean. The values include YES (true) or NO (false).
    • Categorical. Values within a finite set of possible values
    
## Labels
This are the labels corresponding to the probability that the current sample belongs to the given class. This is multilabel problem and hence a given sample can belong to more than one class.


## File descriptions
All the files are CSV.
train.csv - the features x x   of the training set. Each row
corresponds to a different sample, while each column is a different
feature.
    
    • trainLabels.csv - the expected labels y y   for the training set.
            Each row corresponds to a different sample, while each column is a different label. The order of the rows is aligned with train.csv.
            
    • test.csv - the features x x   of the test set.
            Each row corresponds to a different sample, while each column is a different feature.
            
    • sampleSubmission.csv - example of the expected probabilities y ̂  y  ^       for the test set.
            Each row contains two columns, namely one string and the probability of each sample belonging to each label.
            
    For example, if the test.csv has 3 samples and 4 labels, the submission file must have 13 rows with these strings in the first column: id_label, 1_y1, 1_y2, 1_y3, 1_y4, 2_y1, 2_y2, 2_y3, 2_y4, 3_y1, 3_y2, 3_y3, 3_y4, 4_y1, 4_y2, 4_y3, 4_y4
    
    

### Importing the neccessary packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

import time

import hashlib     # Function to hash a string using hashlib

from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import scipy.sparse as sp

from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay, roc_auc_score, roc_curve, RocCurveDisplay

from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier


In [2]:
pd.set_option('display.max_columns', None)

### Reading the datas

In [3]:
# Load training data
train_features = pd.read_csv('./DATA Scientist Assignment/train.csv')
train_labels = pd.read_csv('./DATA Scientist Assignment/trainLabels.csv')

In [4]:
# Inspect the data
train_features.head()

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,YES,NO,NO,NO,NO,42,0.396065,3,6,0.991018,0.0,0.82,3306,4676,YES,NO,YES,0,0.405047,0.464610,NO,NO,NO,NO,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,0.576561,0.073139,0.481394,0.115697,0.458560,YES,NO,YES,NO,NO,9,0.368263,2,10,0.992729,0.0,0.94,3306,4676,YES,NO,YES,1,0.375535,0.451301,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,NO,NO,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,0.576561,0.073139,0.481394,0.115697,0.487598,YES,NO,NO,NO,NO,42,0.363131,6,10,0.987596,0.0,0.71,3306,4676,YES,NO,YES,0,0.375535,0.479734,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,NO,NO,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=,0.576561,0.073139,0.481394,0.115697,0.473079,YES,NO,NO,NO,NO,37,0.333618,4,6,0.987169,0.0,0.89,3306,4676,YES,NO,YES,1,0.346450,0.464610,0.576561,0.073139,0.481394,0.115697,0.473079,YES,NO,NO,NO,NO,42,0.363131,5,6,0.987596,0.0,0.810,3306,4676,YES,NO,YES,2,0.375535,0.464610
1,2,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0,0.000000,0,0,0.000000,0.0,0.00,0,0,NaN,NaN,NaN,0,0.000000,0.000000,NO,NO,NO,NO,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,1.058379,0.125832,0.932547,0.663037,0.569047,YES,NO,NO,NO,NO,9,0.709921,5,6,0.968240,0.0,0.81,4678,3306,YES,NO,YES,3,0.741682,0.560282,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,NO,NO,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,1.058379,0.125832,0.932547,0.663037,0.628474,YES,NO,NO,NO,NO,2,0.679371,8,7,0.937387,0.0,0.84,4678,3306,YES,NO,YES,1,0.741984,0.619282,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,NO,NO,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.058379,0.125832,0.932547,0.663037,0.602394,NO,NO,NO,NO,NO,11,0.581367,3,6,0.966122,0.0,0.87,4678,3306,NO,NO,NO,3,0.615245,0.593630,1.058379,0.125832,0.932547,0.663037,0.602394,YES,NO,NO,NO,NO,9,0.709921,4,6,0.968240,0.0,0.510,4678,3306,YES,NO,YES,4,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,YES,NO,NO,2,0.838475,3,5,0.966122,0.0,0.74,4678,3306,NO,NO,NO,2,0.872353,0.493159,NO,NO,YES,YES,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,1.341803,0.051422,0.935572,0.041440,0.447627,YES,NO,NO,NO,YES,2,0.752269,5,7,0.954930,0.0,0.82,4678,3306,YES,NO,YES,2,0.797338,0.438435,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,NO,NO,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,1.341803,0.051422,0.935572,0.041440,0.522873,YES,NO,NO,NO,NO,1,0.732305,6,6,0.954930,0.0,0.80,4678,3306,YES,NO,YES,0,0.777374,0.513681,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,NO,NO,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,NO,NO,NO,2,0.657290,6,5,0.936479,0.0,0.79,4678,3306,NO,NO,NO,0,0.720811,0.493159,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,YES,NO,NO,5,0.742589,3,5,0.966122,0.0,0.850,4678,3306,NO,NO,NO

In [5]:
train_features.shape

(9999, 146)

In [6]:
# train_features.dtypes.values

In [7]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: float64(55), int64(31), object(60)
memory usage: 11.1+ MB


### Checking null values

In [8]:
train_features.isna().sum().values

array([   0, 1426, 1426, 1426, 1426,    0,    0,    0,    0,    0, 1426,
       1426, 1426, 1426, 1426,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1426, 1426, 1426,    0,    0,    0,    0,    0,  284,
        284,  284,  284,    0,    0,    0,    0,    0,  284,  284,  284,
        284,  284,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        284,  284,  284,    0,    0,    0,    0,  396,  396,  396,  396,
          0,    0,    0,    0,    0,  396,  396,  396,  396,  396,    0,
          0,    0,    0,    0,    0,    0,    0,    0,  396,  396,  396,
          0,    0,    0,    0,  851,  851,  851,  851,    0,    0,    0,
          0,    0,  851,  851,  851,  851,  851,    0,    0,    0,    0,
          0,    0,    0,    0,    0,  851,  851,  851,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int64)

In [9]:
train_features['x1'].value_counts().count()

2

In [10]:
train_features[train_features.duplicated()]

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145


### We can see there's no duplicates.

In [11]:
# Separate numeric, boolean, and categorical columns
numeric_cols = train_features.select_dtypes(include=['number']).columns
boolean_cols = train_features.select_dtypes(include=['bool']).columns
categorical_cols = train_features.select_dtypes(include=['object']).columns

### Getting the binary categorical values

In [16]:

def binary_features(df):
    
    train_features = df
    
    # Identify columns with exactly two unique values
    binary_columns = [col for col in train_features.columns if train_features[col].nunique() == 2]

    # Filter these columns in the DataFrame
    binary_features = train_features[binary_columns]

    # Fill NaN values with 50% 'YES' and 50% 'NO'
    for col in binary_columns:
        num_nans = binary_features[col].isna().sum()
        num_yes = num_nans // 2
        num_no = num_nans - num_yes

        nan_indices = binary_features[binary_features[col].isna()].index
        yes_indices = np.random.choice(nan_indices, num_yes, replace=False)
        no_indices = nan_indices.difference(yes_indices)

        binary_features.loc[yes_indices, col] = 'YES'
        binary_features.loc[no_indices, col] = 'NO'

        #le = LabelEncoder()

        #for col in binary_features.columns:
        #    binary_features[col] = le.fit_transform(binary_features[col])

    # Replace the binary columns in the original DataFrame
    train_features[binary_columns] = binary_features

    return train_features

In [17]:
train_features = binary_features(train_features)

C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_features.loc[yes_indices, col] = 'YES'
C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_features.loc[no_indices, col] = 'NO'
C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_

### Fill 50% of the null values to 'YES' rest 'NO'

### After fill the null values, We can see there's no null values.

In [19]:
train_features.isna().sum().values

array([   0,    0,    0, 1426, 1426,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,  284,  284,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  396,  396,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,  851,  851,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0], dtype=int64)

### Same process for the non binary object datas. In here we fill with null value. 

In [21]:

def multi_category(df):
    
    train_features = df
    
    # Identify columns with exactly two unique values
    multi_columns = [col for col in train_features.select_dtypes(include=['object']).columns if train_features[col].nunique() > 2]

    # Filter these columns in the DataFrame
    multi_features = train_features[multi_columns]

    for col in multi_columns:
        num_nans = multi_features[col].isna().sum()

        nan_indices = multi_features[multi_features[col].isna()].index

        multi_features.loc[nan_indices, col] = 'null'

    # Apply the hash function to the hash columns in both train and test data
    #for col in multi_features.columns:
    #    multi_features[col] = multi_features[col].apply(lambda x: hash_string(str(x)))

    # Replace the binary columns in the original DataFrame
    train_features[multi_columns] = multi_features

    return train_features

In [22]:
train_features = multi_category(train_features)

In [23]:
train_features

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1,NO,NO,dqOiM6yBYgnVSezBRiQXs9bvOFnRqrtIoXRIElxD7g8=,GNjrXXA3SxbgD0dTRblAPO9jFJ7AIaZnu/f48g5XSUk=,0.576561,0.073139,0.481394,0.115697,0.472474,YES,NO,NO,NO,NO,42,0.396065,3,6,0.991018,0.0,0.82,3306,4676,YES,NO,YES,0,0.405047,0.464610,NO,NO,NO,NO,mimucPmJSF6NI6KM6cPIaaVxWaQyIQzSgtwTTb9bKlc=,s7mTY62CCkWUFc36AW2TlYAy5CIcniD2Vz+lHzyYCLg=,0.576561,0.073139,0.481394,0.115697,0.458560,YES,NO,YES,NO,NO,9,0.368263,2,10,0.992729,0.0,0.94,3306,4676,YES,NO,YES,1,0.375535,0.451301,+2TNtXRI6r9owdGCS80Ia9VVv8ZpuOpVaHEvxRGGu78=,NO,NO,Op+X3asn5H7EQJErI7PR0NkUs3YB+Ld/8OfWuiOC8tU=,GeerC2BbPUcQfQO86NmvOsKrfTvmW7HF+Iru9y+7DPA=,0.576561,0.073139,0.481394,0.115697,0.487598,YES,NO,NO,NO,NO,42,0.363131,6,10,0.987596,0.0,0.71,3306,4676,YES,NO,YES,0,0.375535,0.479734,bxU52teuxC05EZyzFihSiKHczE2ZAIVCXekVLG7j3C0=,NO,NO,+dia7tCOijlRGbABX0YKG5L85x/hXLyJwwplN5Qab04=,f4Uu1R9nnf/h03aqiRQT0Fw3WItzNToLCyRlW1Pn8Z8=,0.576561,0.073139,0.481394,0.115697,0.473079,YES,NO,NO,NO,NO,37,0.333618,4,6,0.987169,0.0,0.89,3306,4676,YES,NO,YES,1,0.346450,0.464610,0.576561,0.073139,0.481394,0.115697,0.473079,YES,NO,NO,NO,NO,42,0.363131,5,6,0.987596,0.0,0.810,3306,4676,YES,NO,YES,2,0.375535,0.464610
1,2,YES,NO,null,null,0.000000,0.000000,0.000000,0.000000,0.000000,YES,NO,NO,NO,NO,0,0.000000,0,0,0.000000,0.0,0.00,0,0,NO,NO,YES,0,0.000000,0.000000,NO,NO,NO,NO,l0G2rvmLGE6mpPtAibFsoW/0SiNnAuyAc4k35TrHvoQ=,lblNNeOLanWhqgISofUngPYP0Ne1yQv3QeNHqCAoh48=,1.058379,0.125832,0.932547,0.663037,0.569047,YES,NO,NO,NO,NO,9,0.709921,5,6,0.968240,0.0,0.81,4678,3306,YES,NO,YES,3,0.741682,0.560282,MZZbXga8gvaCBqWpzrh2iKdOkcsz/bG/z4BVjUnqWT0=,NO,NO,TqL9cs8ZFzALzVpZv6wYBDi+6zwhrdarQE/3FH+XAlA=,aZTF/lredyP4cukeN8bh6kpBjYmS1QFNpPOg2LVm3Lg=,1.058379,0.125832,0.932547,0.663037,0.628474,YES,NO,NO,NO,NO,2,0.679371,8,7,0.937387,0.0,0.84,4678,3306,YES,NO,YES,1,0.741984,0.619282,YvZUuCDjLu9VvkCdBWgARWQrvm+FSXgxp0zIrMjcLBc=,NO,NO,dsyhxXKNNJy4WVGD/v4+UGyW3jHWkx2xTdg3STsf34A=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.058379,0.125832,0.932547,0.663037,0.602394,NO,NO,NO,NO,NO,11,0.581367,3,6,0.966122,0.0,0.87,4678,3306,NO,NO,NO,3,0.615245,0.593630,1.058379,0.125832,0.932547,0.663037,0.602394,YES,NO,NO,NO,NO,9,0.709921,4,6,0.968240,0.0,0.510,4678,3306,YES,NO,YES,4,0.741682,0.593630
2,3,NO,NO,ib4VpsEsqJHzDiyL0dZLQ+xQzDPrkxE+9T3mx5fv2wI=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,YES,NO,NO,2,0.838475,3,5,0.966122,0.0,0.74,4678,3306,NO,NO,NO,2,0.872353,0.493159,NO,NO,YES,YES,9TRXThP/ifDpJRGFX1LQseibUA1NJ3XM53gy+1eZ46k=,XSJ6E8aAoZC7/KAu3eETpfMg3mCq7HVBFIVIsoMKh9E=,1.341803,0.051422,0.935572,0.041440,0.447627,YES,NO,NO,NO,YES,2,0.752269,5,7,0.954930,0.0,0.82,4678,3306,YES,NO,YES,2,0.797338,0.438435,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,NO,NO,oFsUwSLCWcj8UA1cqILh5afKVcvwlFA+ohJ147Wkz5I=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,1.341803,0.051422,0.935572,0.041440,0.522873,YES,NO,NO,NO,NO,1,0.732305,6,6,0.954930,0.0,0.80,4678,3306,YES,NO,YES,0,0.777374,0.513681,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,NO,NO,mRPnGiKVOWTk/vzZaqlLXZRtdrkcQ/sX0hqBCqOuKq0=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,NO,NO,NO,2,0.657290,6,5,0.936479,0.0,0.79,4678,3306,NO,NO,NO,0,0.720811,0.493159,1.341803,0.051422,0.935572,0.041440,0.501710,NO,NO,YES,NO,NO,5,0.742589,3,5,0.966122,0.0,0.850,4678,3306,NO,NO,NO,1,0.

In [24]:
train_features.isna().sum()

id      0
x1      0
x2      0
x3      0
x4      0
       ..
x141    0
x142    0
x143    0
x144    0
x145    0
Length: 146, dtype: int64

In [25]:
train_df = train_features.copy()

In [26]:
train_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9999 entries, 0 to 9998
Columns: 146 entries, id to x145
dtypes: float64(55), int64(31), object(60)
memory usage: 11.1+ MB


In [27]:
# getting categorical features from train data...

cat_df = train_features.select_dtypes(include=['object'])

cat_features = train_features.select_dtypes(include=['object']).columns

cat_features

Index(['x1', 'x2', 'x3', 'x4', 'x10', 'x11', 'x12', 'x13', 'x14', 'x24', 'x25',
       'x26', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x41', 'x42', 'x43',
       'x44', 'x45', 'x55', 'x56', 'x57', 'x61', 'x62', 'x63', 'x64', 'x65',
       'x71', 'x72', 'x73', 'x74', 'x75', 'x85', 'x86', 'x87', 'x91', 'x92',
       'x93', 'x94', 'x95', 'x101', 'x102', 'x103', 'x104', 'x105', 'x115',
       'x116', 'x117', 'x126', 'x127', 'x128', 'x129', 'x130', 'x140', 'x141',
       'x142'],
      dtype='object')

### Applyin the TDIDF- Vectorizer

In [29]:
train2 = train_features.copy()

for feature in cat_features:
    # Apply TF-IDF vectorization

    tfidf_vectorizer = TfidfVectorizer(max_features = 2)
    
    tfidf_feature = tfidf_vectorizer.fit_transform(train2[feature])
    
    #print(tfidf_feature)
    
    train2 = train2.drop(columns=[feature])
    
    train2 = pd.concat([pd.DataFrame(tfidf_feature.toarray()), train2], axis=1)

### Scaling the data

In [31]:
train2.columns = train2.columns.astype(str)

std = StandardScaler()

train3 = std.fit_transform(train2)

final_train = pd.DataFrame(train3)

In [32]:
final_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205
0,-0.934813,0.934813,0.14859,-0.14859,-0.949565,0.949565,0.528797,-0.528797,0.160465,-0.160465,0.894971,-0.894971,0.077697,-0.077697,-0.902608,0.902608,-1.464019,1.464019,0.230633,-0.230633,-1.465708,1.465708,0.447777,-0.447777,0.263461,-0.263461,0.770038,-0.770038,0.254891,-0.254891,-1.455289,1.455289,-0.265841,-0.305001,-0.169422,-0.305001,0.308902,-0.308902,0.381950,-0.381950,-0.235167,-0.235167,-1.268599,1.268599,0.189608,-0.189608,-1.276146,1.276146,0.368771,-0.368771,0.181534,-0.181534,0.643495,-0.643495,0.174355,-0.174355,-1.250276,1.250276,-0.233266,-0.233266,-0.203069,-0.193535,0.290874,-0.290874,0.351222,-0.351222,-0.191861,-0.191861,-1.322590,1.322590,0.168168,-0.168168,-1.331495,1.331495,0.364558,-0.364558,0.169109,-0.169109,-1.527416,1.527416,0.162096,-0.162096,-1.305053,1.305053,-0.229428,-0.229428,-0.184742,-0.184742,0.276049,-0.276049,0.339980,-0.339980,0.236348,-0.236348,0.288847,-0.288847,-1.250538,1.250538,0.2961,-0.2961,-1.253171,1.253171,0.486397,-0.486397,0.322692,-0.322692,0.786272,-0.786272,0.31661,-0.31661,-1.239309,1.239309,-0.226759,-0.407843,-0.173134,-0.407843,0.375910,-0.375910,0.429459,-0.429459,-1.731878,-0.740232,0.151886,-0.878525,-0.173273,0.081133,4.104955,-0.076479,-0.338037,-0.294927,0.499739,0.128467,0.090058,0.579277,1.864294,-0.691244,-0.153452,0.083726,-1.230147,0.104771,-1.846781,-0.202356,-0.097083,0.078264,-0.226732,-0.536621,0.269334,0.435993,0.149883,0.232379,0.438832,1.833199,-0.392065,-0.399425,-0.087479,-1.222908,0.096953,-1.667531,-0.238825,-0.100812,3.195673,-0.216962,-0.127757,0.252283,0.415496,0.135713,-0.792186,0.445374,1.829111,-0.654121,-0.378134,-0.094738,-0.952283,0.144691,-1.209777,-0.199557,-0.013653,2.926908,-0.117194,-0.186084,-0.346482,0.407124,0.133224,0.161707,0.503857,1.837564,-0.349905,-0.200807,-0.011686,-1.475663,0.087135,-2.690826,-0.233831,-0.168363,4.064889,-0.262241,-0.566444,-0.425390,0.769844,-0.037165,-0.968699,0.400915,1.826982,-0.151420,-0.531420,-0.157912
1,-0.934813,0.934813,0.14859,-0.14859,-0.949565,0.949565,0.528797,-0.528797,0.160465,-0.160465,0.894971,-0.894971,0.077697,-0.077697,-0.902608,0.902608,0.683051,-0.683051,0.230633,-0.230633,0.682264,-0.682264,0.447777,-0.447777,0.263461,-0.263461,0.770038,-0.770038,0.254891,-0.254891,0.687149,-0.687149,-0.265841,-0.305001,-0.169422,-0.305001,0.308902,-0.308902,0.381950,-0.381950,-0.235167,-0.235167,-1.268599,1.268599,0.189608,-0.189608,-1.276146,1.276146,0.368771,-0.368771,0.181534,-0.181534,0.643495,-0.643495,0.174355,-0.174355,-1.250276,1.250276,-0.233266,-0.233266,-0.203069,-0.193535,0.290874,-0.290874,0.351222,-0.351222,-0.191861,-0.191861,-1.322590,1.322590,0.168168,-0.168168,-1.331495,1.331495,0.364558,-0.364558,0.169109,-0.169109,0.654700,-0.654700,0.162096,-0.162096,-1.305053,1.305053,-0.229428,-0.229428,-0.184742,-0.184742,0.276049,-0.276049,0.339980,-0.339980,0.236348,-0.236348,0.288847,-0.288847,-1.250538,1.250538,0.2961,-0.2961,0.797976,-0.797976,0.486397,-0.486397,0.322692,-0.322692,0.786272,-0.786272,0.31661,-0.31661,-1.239309,1.239309,-0.226759,2.451922,-0.173134,2.451922,0.375910,-0.375910,-2.328511,2.328511,-1.731531,-1.839152,-0.417309,-2.241569,-0.524932,-1.490985,-0.691728,-1.421060,-1.021654,-1.148179,-2.428892,0.128467,-2.300997,-1.322629,-1.238706,-0.691244,-1.495225,-1.470

### Slicing train labels to obtain the required train labels for the given 9,999 train data

In [33]:
# slicing train labels to obtain the required train labels for the given 9,999 train data

temp = train_labels[:9999]
y = temp.drop(['id'], axis =1)
y

,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
9996,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Splitting the data for training and validation

In [34]:
X_train, X_test, y_train, y_test = train_test_split(final_train, y, test_size=0.3, random_state=3)

### Applying GridSearchCV to find the best parameter of the RandomForestClassifier model

In [35]:
params = {'n_estimators' : [100, 500], 'max_depth' : [15, 20]}

model = RandomForestClassifier()

clf = GridSearchCV(model,params, cv= 3, scoring='f1_micro', return_train_score=True, n_jobs = -1)

clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'max_depth': [15, 20], 'n_estimators': [100, 500]},
             return_train_score=True, scoring='f1_micro')

### Here's the best parameters

In [36]:
print(clf.best_params_)

{'max_depth': 20, 'n_estimators': 500}


### Defining the RFC with the best parameters

In [37]:
v_rfc = RandomForestClassifier(max_depth=20, n_estimators=500)
#v_rfc = RandomForestClassifier(n_estimators=10, criterion='entropy')

v_rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, n_estimators=500)

### After train the model, applying to prediction

In [38]:
v_rfc_predict = v_rfc.predict(X_test)

In [39]:
v_rfc_predict

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

### After prediction, Check the accuracy score

In [40]:
print('Accuracy of the training set: ', accuracy_score(y_test, v_rfc_predict))

Accuracy of the training set:  0.722


### Classification report

In [41]:
print('Classification report of the training set: \n', classification_report(y_test, v_rfc_predict))

Classification report of the training set: 
               precision    recall  f1-score   support

           0       1.00      0.09      0.17        22
           1       0.00      0.00      0.00         2
           2       0.98      0.60      0.75        68
           3       1.00      0.84      0.91        43
           4       0.00      0.00      0.00         1
           5       0.89      0.60      0.72       239
           6       1.00      0.31      0.47       116
           7       0.00      0.00      0.00         2
           8       0.95      0.62      0.75       223
           9       0.90      0.41      0.57        46
          10       0.00      0.00      0.00         1
          11       0.87      0.55      0.67       233
          12       1.00      0.42      0.60        40
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         5
          15       0.92      0.38      0.53        32
          16       0.00      0.00   

C:\Users\johan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\johan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\johan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie

### Same process for the test set

In [43]:
# Load test data
test_features = pd.read_csv('./DATA Scientist Assignment/test.csv')

In [44]:
test = pd.DataFrame(test_features.values, columns=train_features.columns)

In [45]:
test

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,3,0.8955,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,1,0.945032,0.471747,NO,NO,NO,NO,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,0.832679,0.049834,0.945938,0.317427,0.480308,NO,NO,YES,NO,NO,4,0.804893,3,7,0.966777,0.0,0.84,4672,3311,NO,NO,NO,3,0.838115,0.471318,6Wv/YGbS0KDFv2UrATvlJcVtCjJOgnVQzWuuF5Ltv2k=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,dBSc/QZM58O6miC4ULLhY0C4S6WIZLwy2oERlRo7Iaw=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,0,0.725763,17,7,0.876992,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
1,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,YES,YES,YES,4,0.536996,2,10,0.979821,0.0,1.0,1263,892,NO,NO,NO,8,0.557175,0.693587,NO,NO,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,0+HOWI7C643OXIZ7LdoeNFzA7A0MkSH//diziszbII8=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,eI6XNOFa+S9J5wTICUEas3M6YE4TJ6rSTCo2NvqM16s=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,1.415919,0.0,1.0,0.0,0.703088,NO,NO,YES,YES,YES,0,0.466368,9,10,0.919283,-1.0,1.0,1263,892,NO,NO,NO,8,0.557175,0.693587
2,1698004,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0,0.0,0,0,0.0,0.0,0.0,0,0,NaN,NaN,NaN,0,0.0,0.0,NO,NO,NO,NO,+X/rq+4qeKy2UVQN5xLQHSbri0Vdd/mYdG7WH/Kn+Pg=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,0.646703,0.059891,0.92196,0.53176,0.386558,NO,NO,NO,NO,NO,2,0.771325,6,9,0.939504,0.0,0.88,4672,3306,NO,NO,NO,3,0.831821,0.377568,nxOTdokWdl63qtAt4sgkSvyREdGu+ht1Iw4pcCaMi64=,YES,NO,QuSoZ63+jzXv3Ux1Y2LB+aSj7QCdPbrup3JSBj6Dmx8=,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,0.646703,0.059891,0.92196,0.53176,0.427226,YES,NO,YES,NO,YES,3,0.833031,1,7,0.992136,0.0,0.93,4672,3306,YES,NO,YES,1,0.840895,0.419949,B+EJpnEbkYtLnwDQYN1dP1rcfnoCnxAjKLYwQZE07Ew=,NO,NO,JA7Y7rY1wqr46lhqu5scuWov+Xw4neBTlPOPObhG0V8=,PjdztVAv+7nhxs8/+uAT0IRL7OcEHhLwHZ0IDTVbqcs=,0.646703,0.059891,0.92196,0.53176,0.416096,NO,NO,NO,NO,NO,13,0.161525,25,3,0.749546,0.0,0.86,4672,3306,NO,NO,NO,1,0.411978,0.405394,0.646703,0.059891,0.92196,0.53176,0.413099,YES,NO,NO,NO,NO,2,0.823351,5,3,0.952813,0.0,0.87,4672,3306,YES,NO,YES,0,0.870538,0.405822
3,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.1

In [46]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1999 entries, 0 to 1998
Columns: 146 entries, id to x145
dtypes: object(146)
memory usage: 2.2+ MB


In [47]:
### There's no duplicate values.

In [48]:
test_df = test.copy()

In [49]:
test_df.isna().sum().values

array([  0, 294, 294, 294, 294,   0,   0,   0,   0,   0, 294, 294, 294,
       294, 294,   0,   0,   0,   0,   0,   0,   0,   0,   0, 294, 294,
       294,   0,   0,   0,   0,   0,  73,  73,  73,  73,   0,   0,   0,
         0,   0,  73,  73,  73,  73,  73,   0,   0,   0,   0,   0,   0,
         0,   0,   0,  73,  73,  73,   0,   0,   0,   0,  81,  81,  81,
        81,   0,   0,   0,   0,   0,  81,  81,  81,  81,  81,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  81,  81,  81,   0,   0,   0,
         0, 171, 171, 171, 171,   0,   0,   0,   0,   0, 171, 171, 171,
       171, 171,   0,   0,   0,   0,   0,   0,   0,   0,   0, 171, 171,
       171,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0], dtype=int64)

In [50]:
test_features = binary_features(test_df)

C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_features.loc[yes_indices, col] = 'YES'
C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_features.loc[no_indices, col] = 'NO'
C:\Users\johan\AppData\Local\Temp\ipykernel_19840\2391711323.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary_

###  Treating Hash fields
    It's not possible to "dehash" or reverse a cryptographic hash function like SHA-256 back to its original text form using Python or any other programming language.
    This is because cryptographic hash functions are designed to be one-way, meaning they are easy to compute in one direction but extremely difficult to reverse.

In [52]:
test_features = multi_category(test_df)

In [53]:
test_features

,id,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18,x19,x20,x21,x22,x23,x24,x25,x26,x27,x28,x29,x30,x31,x32,x33,x34,x35,x36,x37,x38,x39,x40,x41,x42,x43,x44,x45,x46,x47,x48,x49,x50,x51,x52,x53,x54,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64,x65,x66,x67,x68,x69,x70,x71,x72,x73,x74,x75,x76,x77,x78,x79,x80,x81,x82,x83,x84,x85,x86,x87,x88,x89,x90,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100,x101,x102,x103,x104,x105,x106,x107,x108,x109,x110,x111,x112,x113,x114,x115,x116,x117,x118,x119,x120,x121,x122,x123,x124,x125,x126,x127,x128,x129,x130,x131,x132,x133,x134,x135,x136,x137,x138,x139,x140,x141,x142,x143,x144,x145
0,1698002,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,3,0.8955,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,1,0.945032,0.471747,NO,NO,NO,NO,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,0.832679,0.049834,0.945938,0.317427,0.480308,NO,NO,YES,NO,NO,4,0.804893,3,7,0.966777,0.0,0.84,4672,3311,NO,NO,NO,3,0.838115,0.471318,6Wv/YGbS0KDFv2UrATvlJcVtCjJOgnVQzWuuF5Ltv2k=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,dBSc/QZM58O6miC4ULLhY0C4S6WIZLwy2oERlRo7Iaw=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,1,0.725763,6,7,0.950468,0.0,0.88,4672,3311,YES,NO,YES,0,0.775294,0.471747,0.832679,0.049834,0.945938,0.317427,0.482021,YES,NO,NO,NO,NO,0,0.725763,17,7,0.876992,1.0,0.866667,4672,3311,NO,NO,NO,5,0.945032,0.471318
1,1698003,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,YES,YES,YES,4,0.536996,2,10,0.979821,0.0,1.0,1263,892,NO,NO,NO,8,0.557175,0.693587,NO,NO,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,0+HOWI7C643OXIZ7LdoeNFzA7A0MkSH//diziszbII8=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,eI6XNOFa+S9J5wTICUEas3M6YE4TJ6rSTCo2NvqM16s=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,1.415919,0.0,1.0,0.0,0.703088,NO,NO,NO,NO,NO,3,0.466368,6,10,0.939462,0.0,1.0,1263,892,NO,NO,NO,7,0.526906,0.693587,1.415919,0.0,1.0,0.0,0.703088,NO,NO,YES,YES,YES,0,0.466368,9,10,0.919283,-1.0,1.0,1263,892,NO,NO,NO,8,0.557175,0.693587
2,1698004,NO,NO,null,null,0.0,0.0,0.0,0.0,0.0,YES,YES,YES,NO,YES,0,0.0,0,0,0.0,0.0,0.0,0,0,NO,NO,NO,0,0.0,0.0,NO,NO,NO,NO,+X/rq+4qeKy2UVQN5xLQHSbri0Vdd/mYdG7WH/Kn+Pg=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,0.646703,0.059891,0.92196,0.53176,0.386558,NO,NO,NO,NO,NO,2,0.771325,6,9,0.939504,0.0,0.88,4672,3306,NO,NO,NO,3,0.831821,0.377568,nxOTdokWdl63qtAt4sgkSvyREdGu+ht1Iw4pcCaMi64=,YES,NO,QuSoZ63+jzXv3Ux1Y2LB+aSj7QCdPbrup3JSBj6Dmx8=,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,0.646703,0.059891,0.92196,0.53176,0.427226,YES,NO,YES,NO,YES,3,0.833031,1,7,0.992136,0.0,0.93,4672,3306,YES,NO,YES,1,0.840895,0.419949,B+EJpnEbkYtLnwDQYN1dP1rcfnoCnxAjKLYwQZE07Ew=,NO,NO,JA7Y7rY1wqr46lhqu5scuWov+Xw4neBTlPOPObhG0V8=,PjdztVAv+7nhxs8/+uAT0IRL7OcEHhLwHZ0IDTVbqcs=,0.646703,0.059891,0.92196,0.53176,0.416096,NO,NO,NO,NO,NO,13,0.161525,25,3,0.749546,0.0,0.86,4672,3306,NO,NO,NO,1,0.411978,0.405394,0.646703,0.059891,0.92196,0.53176,0.413099,YES,NO,NO,NO,NO,2,0.823351,5,3,0.952813,0.0,0.87,4672,3306,YES,NO,YES,0,0.870538,0.405822
3,1698005,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,1.12921

### Getting the Non numerical columns

In [57]:
test_df_cat1 = test_features[['x1', 'x2', 'x3', 'x4', 'x10', 'x11', 'x12', 'x13', 'x14', 'x24', 'x25',
       'x26', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x41', 'x42', 'x43',
       'x44', 'x45', 'x55', 'x56', 'x57', 'x61', 'x62', 'x63', 'x64', 'x65',
       'x71', 'x72', 'x73', 'x74', 'x75', 'x85', 'x86', 'x87', 'x91', 'x92',
       'x93', 'x94', 'x95', 'x101', 'x102', 'x103', 'x104', 'x105', 'x115',
       'x116', 'x117', 'x126', 'x127', 'x128', 'x129', 'x130', 'x140', 'x141',
       'x142']].copy()

In [58]:
test_df_cat1

,x1,x2,x3,x4,x10,x11,x12,x13,x14,x24,x25,x26,x30,x31,x32,x33,x34,x35,x41,x42,x43,x44,x45,x55,x56,x57,x61,x62,x63,x64,x65,x71,x72,x73,x74,x75,x85,x86,x87,x91,x92,x93,x94,x95,x101,x102,x103,x104,x105,x115,x116,x117,x126,x127,x128,x129,x130,x140,x141,x142
0,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,cr+kkNnNFV9YL0vz029hk3ohIDmGuABRVNhFe0ePZyo=,X6dDAI/DZOWvu0Dg6gCgRoNr2vTUz/mc4SdHTNUPS38=,NO,NO,YES,NO,NO,NO,NO,NO,6Wv/YGbS0KDFv2UrATvlJcVtCjJOgnVQzWuuF5Ltv2k=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,YES,NO,NO,NO,NO,YES,NO,YES,dBSc/QZM58O6miC4ULLhY0C4S6WIZLwy2oERlRo7Iaw=,NO,NO,9ACcuXc7MMm9V7jZSr3P3VxAKyMvLAtsdwPKwgncc+k=,WV5vAHFyqkeuyFB5KVNGFOBuwjkUGKYc8wh9QfpVzAA=,YES,NO,NO,NO,NO,YES,NO,YES,YES,NO,NO,NO,NO,NO,NO,NO
1,NO,NO,MeBJ/ZzEIXfNKat4w1oeDxiMNKrAeY0PH41i00hpYDo=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,NO,NO,YES,YES,YES,NO,NO,NO,NO,NO,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,NO,NO,NO,NO,NO,NO,NO,NO,0+HOWI7C643OXIZ7LdoeNFzA7A0MkSH//diziszbII8=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,NO,NO,NO,NO,NO,NO,NO,NO,eI6XNOFa+S9J5wTICUEas3M6YE4TJ6rSTCo2NvqM16s=,NO,NO,W4WKBYyqy7COFohWfUyg2xuUPldk8b4hPsDCHjbePaU=,ABhwE8nsQMtFix2MDemgmGfoV68Tn3hLpZRXWVTQ0TM=,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,YES,YES,YES,NO,NO,NO
2,NO,NO,null,null,YES,YES,YES,NO,YES,NO,NO,NO,NO,NO,NO,NO,+X/rq+4qeKy2UVQN5xLQHSbri0Vdd/mYdG7WH/Kn+Pg=,oo9tGpHvTredpg9JkHgYbZAuxcwtSpQxU5mA/zUbxY8=,NO,NO,NO,NO,NO,NO,NO,NO,nxOTdokWdl63qtAt4sgkSvyREdGu+ht1Iw4pcCaMi64=,YES,NO,QuSoZ63+jzXv3Ux1Y2LB+aSj7QCdPbrup3JSBj6Dmx8=,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,YES,NO,YES,NO,YES,YES,NO,YES,B+EJpnEbkYtLnwDQYN1dP1rcfnoCnxAjKLYwQZE07Ew=,NO,NO,JA7Y7rY1wqr46lhqu5scuWov+Xw4neBTlPOPObhG0V8=,PjdztVAv+7nhxs8/+uAT0IRL7OcEHhLwHZ0IDTVbqcs=,NO,NO,NO,NO,NO,NO,NO,NO,YES,NO,NO,NO,NO,YES,NO,YES
3,NO,NO,uduY7XWJ8eFgTltv5P0rPh5GW6KwBu+tPFH13uQRN+0=,0L7+hNDV8S57etySgdljbm2AK1zQuLP77lGk2hyEmCo=,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,NO,j3r3dqveRaSq13FDqUkrD+rGAW5DmYjnfEdKfVic3Do=,tnLDGLnpYhzsik5+X+WPo4KQJoQA0TfWRlmEtQ3XNJQ=,NO,NO,YES,NO,YES,NO,NO,NO,8Whd23AFTt1KV61HEnaVzYZCSZsw5sqqmf4WUmWd3bQ=,NO,NO,IL4RpsAr/2F3DyhivH6T3zNTVSv14noGCEuy6VR0R0A=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,NO,NO,YES,NO,NO,NO,NO,NO,+yhSY//Hpg7u0bSA7NYmcmRFgv3bF4Tw3BMHrBqaTtA=,NO,NO,ZWvUbzI3yDZ8UGVOrIfsQCDpIBme7hoxGV079DPM78g=,ubHy/++3FVAS97znZOt7L+cjkZFJREIiJPRZEfRIztc=,NO,NO,YES,NO,NO,NO,NO,NO,YES,NO,YES,NO,YES,YES,NO,YES
4,NO,NO,kM4KU87XvnvKRvf4dN3Tu4zQYq8fpcqhDTFADWdfCg8=,4LhhvTzxwvh2SnFtcpaRasyvph66a3YDIQCshAfyS2o=,NO,NO,YES,NO,NO,NO,NO,NO,NO,NO,NO,NO,M4ra4ZsGWR+veMAUMfpKnI4R8f5lQmz332MQ1RAcPGY=,BWlzsfzvLpUVVqvMBbjZ4zlrnQb/agQ7zCXv27i3RUw=,YES,NO,NO,NO,YES,YES,NO,YES,sFxo2PpANwP+wYs1Jr0X/Mj94f3w/wzTeq8mKe0DrE8=,NO,NO,M4ra4ZsGWR+veMAUMfpKnI4R8f5lQmz332MQ1RAcPGY=,BWlzsfzvLpUVVqvMBbjZ4zlrnQb/agQ7zCXv27i3RUw=,YES,NO,NO,NO,YES,YES,NO,YES,A0qW1vORvEPazvD0/mLe06Hm//wfSuxG8qBnFBj8RlI=,NO,NO,M4ra4ZsGWR+veMAUMfpKnI4R8f5lQmz332MQ1RAcPGY=,BWlzsfzvLpUVVqvMBbjZ4zlrnQb/agQ7zCXv27i3RUw=,YES,NO,NO,NO,YES,YES,NO,YES,YES,NO,YES,NO,YES,YES,NO,YES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,NO,NO,Z6vucL/W0MPoFsgu2ewNXrvNCAQFiKzUJTYuqh6lP28=,yhI9Bw5Q8l1vEll4sw/Tem/jojpE9KwjKvQQIyrAqgU=,YES,NO,NO,NO,NO,YES,NO,YES,NO,NO,NO,NO,J0BXf75QysCCr1LQvwLwqKywUU/ARUnFbGjg+QdLpX8=,ilf2dGT7f31KRWK57leidnw87VyQr7sdJUl+Ffqrnjs=,YES,NO,NO,NO,YES,YES,NO,YES,XwGF4l3BgP/6u3gHERvOObKrF89JjyzD07RwwVRnm8E=,NO,NO,J0BXf75QysCCr1LQvwLwqKywUU/ARUnFbGjg+QdLpX8=,ilf2dGT7f31KRWK57leidnw87VyQr7sdJUl+Ffqrnjs=,YES,NO,NO,NO,YES,YES,NO,YES,8W2jqgU9VhlI3ledX0DtFS20qRfWG4lMiGtXVF629RE=,NO,NO,J0BXf75QysCCr1LQvwLwqKywUU/ARUnFbGjg+Q

In [61]:
test_df_cat1.isna().sum().values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

### Applying TF-IDF Vectorizer to the test set

In [63]:
test2 = test_features.copy()

for feature in test_df_cat1.columns:
    
    # Apply TF-IDF vectorization
    tfidf_vectorizer = TfidfVectorizer(max_features = 2)
    
    tfidf_feature = tfidf_vectorizer.fit_transform(test2[feature])
    
    test2 = test2.drop(columns=[feature])
    
    test2 = pd.concat([pd.DataFrame(tfidf_feature.toarray()), test2], axis=1)

### Converting the datatype into float. After this step all the columns are numerical.

In [81]:
# test3 = test2.select_dtypes(include=['object']).astype('float64')

test3 = test2.astype('float64')

### Scaling the data

In [83]:
test3.columns = test3.columns.astype(str)

test3_std = std.fit_transform(test3)

In [84]:
final_test = pd.DataFrame(test3_std)

### Make predictions to the test set.

In [86]:
test_predict = v_rfc.predict(final_test)

In [87]:
test_predict

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=int64)

### Converting the predictions to dataframe.

In [88]:
test_predict_df = pd.DataFrame(test_predict, columns = y.columns)

In [92]:
test_predict_df = pd.concat([test_features['id'], test_predict_df], axis=1)

In [93]:
test_predict_df

,id,y1,y2,y3,y4,y5,y6,y7,y8,y9,y10,y11,y12,y13,y14,y15,y16,y17,y18,y19,y20,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30,y31,y32,y33
0,1698002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1698003,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1698004,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1698005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1698006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,1699996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1995,1699997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1996,1699998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1997,1699999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


### Getting the id column values and the prediction columns, and make new column that's 'id_label" and 'Pred'.

In [115]:
id_vars = test_predict_df.columns[0]  # 'A' column
value_vars = test_predict_df.columns[1:]  # 'B', 'C', 'D' columns

melted_df = pd.melt(test_predict_df, id_vars=id_vars, value_vars=value_vars, var_name='label', value_name='Pred')

melted_df['id_label'] = melted_df['id'].astype(str)+'_'+melted_df['label']

melted_df = melted_df.drop(['id', 'label'], axis=1)

melted_df = melted_df[['id_label', 'Pred']]

### Exporting the final results in Excel.

In [117]:
melted_df.to_excel('Dileep_submission.xlsx', index=False)

### After reading the excel display the sample results.

In [130]:
pd.read_excel('Dileep_submission.xlsx').sample(10)

,id_label,Pred
22534,1698547_y12,1
51046,1699073_y26,0
37501,1699521_y19,0
32659,1698677_y17,0
8089,1698095_y5,0
9024,1699030_y5,0
13006,1699014_y7,0
22405,1698418_y12,0
37592,1699612_y19,0
4010,1698014_y3,0
